In [1]:
import os
import gc
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm
from scipy.signal import butter, lfilter, medfilt

%matplotlib qt
sns.set(font_scale=1.5)
sns.set_style("white", {'font.family':'serif', 'font.serif':'Times New Roman'})

# Constants

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

# Filter

In [3]:
class LowPassFilter(object):
    @staticmethod
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=6, fs=1500, order=2, axis=-1):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data, axis=axis)
        y = y - np.median(y)
        return y


# Functions

In [4]:
def standardize(x):
    # return (x - np.mean(x, axis=0)) / np.std(x, axis=0)
    return (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))

def get_processed_grf(grf, clip=False):
    singn_corrected = grf # if np.mean(grf) > 20 else (-grf)
    processed_grf = LowPassFilter.apply(singn_corrected, fs=1500, cutoff=15)
    processed_grf = processed_grf - np.median(processed_grf)
    # processed_grf = grf
    if clip:
        processed_grf[processed_grf < FORCE_THRESHOLD] = 0

    # return processed_grf
    return resample(processed_grf, N_SAMPLES * 3, axis=0).T

def get_processed_moment(moment):
    # processed_moment = moment
    processed_moment = LowPassFilter.apply(moment, fs=1500, cutoff=15)

    # return processed_moment
    return resample(processed_moment, N_SAMPLES * 3, axis=0).T

def get_trajectories(content, foot='right'):
    if foot == 'right':
        trajectory_x = content['data']['points'][0, 9:15, :]
        trajectory_y = content['data']['points'][1, 9:15, :]
        trajectory_z = content['data']['points'][2, 9:15, :]

    elif foot == 'left':
        trajectory_x = content['data']['points'][0, 20:26, :]
        trajectory_y = content['data']['points'][1, 20:26, :]
        trajectory_z = content['data']['points'][2, 20:26, :]

    else:
        raise ValueError('Third foot is not allowed')

    # trajectory_x = LowPassFilter.apply(trajectory_x, fs=100, cutoff=6, order=4)
    # trajectory_y = LowPassFilter.apply(trajectory_y, fs=100, cutoff=6, order=4)
    # trajectory_z = LowPassFilter.apply(trajectory_z, fs=100, cutoff=6, order=4)

    # trajectory_x = resample(trajectory_x, N_SAMPLES * 3, axis=1).T
    # trajectory_y = resample(trajectory_y, N_SAMPLES * 3, axis=1).T
    # trajectory_z = resample(trajectory_z, N_SAMPLES * 3, axis=1).T

    return trajectory_x, trajectory_y, trajectory_z

def get_force(content):
    force_x = []
    force_y = []
    force_z = []
    for i in range(2):
        force_x.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][1, :]
            )
        )
        force_y.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][0, :]
            )
        )
        force_z.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][2, :], True
            )
        )

    return force_x, force_y, force_z

def get_moment(content):
    moment_x = []
    moment_y = []
    moment_z = []
    for i in range(2):
        moment_x.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][1, :]
            )
        )
        moment_y.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][0, :]
            )
        )
        moment_z.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][2, :]
            )
        )

    return moment_x, moment_y, moment_z

# Extract Features

In [5]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

records = []
n_samples = 0
n_bad_samples = 0

# subjects = subjects[:18]

plt.figure(figsize=(8, 6))
# ax = plt.axes(projection='3d')

for subject in tqdm(subjects):
    #     print(subject)
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        if "ST" in filename:
            continue

        if "C4" not in filename:
            continue

        # print(subject, filename, end=' ')

        try:
            content = c3d(
                os.path.join(path, subject, filename), extract_forceplat_data=True
            )

            # print('loaded', end=' ')

            trajectory_x_r, trajectory_y_r, trajectory_z_r = get_trajectories(
                content, "right"
            )
            trajectory_x_l, trajectory_y_l, trajectory_z_l = get_trajectories(
                content, "left"
            )

            force_x, force_y, force_z = get_force(content)
            moment_x, moment_y, moment_z = get_moment(content)

            # plt.plot(moment_x[0])
            # plt.plot(moment_x[1])
            # ax.plot3D(trajectory_y_r[3, 50:], trajectory_x_r[3, 50:], trajectory_z_r[3, 50:])

            del content
            gc.collect()

            force_combined_z = force_z[0] + force_z[1]
            force_mask = force_combined_z >= FORCE_THRESHOLD

            # ... Remove all zero GRF samples
            if (
                np.sum(force_z[0]) <= FORCE_THRESHOLD
                or np.sum(force_z[1]) <= FORCE_THRESHOLD
            ):
                # print(os.path.join(path, subject, filename))
                n_bad_samples = n_bad_samples + 1
                continue

            # ... Remove bad samples
            if np.mean(force_mask) <= 0.5:
                # print(os.path.join(path, subject, filename))
                n_bad_samples = n_bad_samples + 1
                continue

            # ... Remove samples that starts and ends abruptly
            if (
                np.mean(force_combined_z[:10]) >= 200
                or np.mean(force_combined_z[-10:]) >= 200
            ):
                # print(os.path.join(path, subject, filename))
                n_bad_samples = n_bad_samples + 1
                continue

            mask_force_z_1 = force_z[0] > FORCE_THRESHOLD
            mask_force_z_2 = force_z[1] > FORCE_THRESHOLD

            start_1 = np.min(np.argwhere(mask_force_z_1 == True))
            end_1 = np.max(np.argwhere(mask_force_z_1 == True))

            if (end_1 - start_1) < 700 or (end_1 - start_1) > 1700:
                n_bad_samples = n_bad_samples + 1
                continue

            start_2 = np.min(np.argwhere(mask_force_z_2 == True))
            end_2 = np.max(np.argwhere(mask_force_z_2 == True))

            if (end_2 - start_2) < 700 or (end_2 - start_2) > 1700:
                n_bad_samples = n_bad_samples + 1
                continue

            # print('1', end=' ')

            force_x_1 = force_x[0][start_1:end_1]
            force_y_1 = force_y[0][start_1:end_1]
            force_z_1 = force_z[0][start_1:end_1]

            force_x_2 = force_x[1][start_2:end_2]
            force_y_2 = force_y[1][start_2:end_2]
            force_z_2 = force_z[1][start_2:end_2]

            moment_x_1 = moment_x[0][start_1:end_1]
            moment_y_1 = moment_y[0][start_1:end_1]
            moment_z_1 = moment_z[0][start_1:end_1]

            moment_x_2 = moment_x[1][start_2:end_2]
            moment_y_2 = moment_y[1][start_2:end_2]
            moment_z_2 = moment_z[1][start_2:end_2]

            temp_l = trajectory_z_l[start_1:end_1, 0]
            temp_r = trajectory_z_r[start_1:end_1, 0]

            force_x_r = 0
            force_y_r = 0
            force_z_r = 0

            force_x_l = 0
            force_y_l = 0
            force_z_l = 0

            moment_x_r = 0
            moment_y_r = 0
            moment_z_r = 0

            moment_x_l = 0
            moment_y_l = 0
            moment_z_l = 0

            if np.sum(temp_l) > np.sum(temp_r):
                force_x_r = force_x_1
                force_y_r = force_y_1
                force_z_r = force_z_1

                force_x_l = force_x_2
                force_y_l = force_y_2
                force_z_l = force_z_2

                moment_x_r = moment_x_1
                moment_y_r = moment_y_1
                moment_z_r = moment_z_1

                moment_x_l = moment_x_2
                moment_y_l = moment_y_2
                moment_z_l = moment_z_2

                trajectory_x_r = trajectory_x_l[start_2:end_2]
                trajectory_y_r = trajectory_y_r[start_2:end_2]
                trajectory_z_r = trajectory_z_r[start_2:end_2]

                trajectory_x_l = trajectory_x_r[start_1:end_1]
                trajectory_y_l = trajectory_y_l[start_1:end_1]
                trajectory_z_l = trajectory_z_l[start_1:end_1]

            else:
                force_x_l = force_x_1
                force_y_l = force_y_1
                force_z_l = force_z_1

                force_x_r = force_x_2
                force_y_r = force_y_2
                force_z_r = force_z_2

                moment_x_l = moment_x_1
                moment_y_l = moment_y_1
                moment_z_l = moment_z_1

                moment_x_r = moment_x_2
                moment_y_r = moment_y_2
                moment_z_r = moment_z_2

                trajectory_x_r = trajectory_x_l[start_1:end_1]
                trajectory_y_r = trajectory_y_r[start_1:end_1]
                trajectory_z_r = trajectory_z_r[start_1:end_1]

                trajectory_x_l = trajectory_x_r[start_2:end_2]
                trajectory_y_l = trajectory_y_l[start_2:end_2]
                trajectory_z_l = trajectory_z_l[start_2:end_2]

            # print('2', end=' ')

            # ... Resample
            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=0)
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=0)
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=0)

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=0)
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=0)
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=0)

            force_x_r = resample(force_x_r, N_SAMPLES, axis=0)
            force_y_r = resample(force_y_r, N_SAMPLES, axis=0)
            force_z_r = resample(force_z_r, N_SAMPLES, axis=0)

            force_x_l = resample(force_x_l, N_SAMPLES, axis=0)
            force_y_l = resample(force_y_l, N_SAMPLES, axis=0)
            force_z_l = resample(force_z_l, N_SAMPLES, axis=0)

            moment_x_r = resample(moment_x_r, N_SAMPLES, axis=0)
            moment_y_r = resample(moment_y_r, N_SAMPLES, axis=0)
            moment_z_r = resample(moment_z_r, N_SAMPLES, axis=0)

            moment_x_l = resample(moment_x_l, N_SAMPLES, axis=0)
            moment_y_l = resample(moment_y_l, N_SAMPLES, axis=0)
            moment_z_l = resample(moment_z_l, N_SAMPLES, axis=0)

            # print('3', end=' ')

            # ... sign correction
            if np.mean(force_y_r[: int(N_SAMPLES / 2)]) > 0:
                force_y_r = -force_y_r
            if np.mean(force_y_l[: int(N_SAMPLES / 2)]) > 0:
                force_y_l = -force_y_l

            if np.mean(force_x_r) < 0:
                force_x_r = -force_x_r
            if np.mean(force_x_l) < 0:
                force_x_l = -force_x_l

            if np.mean(moment_y_r) < 0:
                moment_y_r = -moment_y_r
            if np.mean(moment_y_l) < 0:
                moment_y_l = -moment_y_l

            if np.mean(moment_x_r[: int(N_SAMPLES / 2)]) > np.mean(moment_x_r[int(N_SAMPLES / 2):]):
                moment_x_r = -moment_x_r
            if np.mean(moment_x_l[: int(N_SAMPLES / 2)]) > np.mean(moment_x_l[int(N_SAMPLES / 2):]):
                moment_x_l = -moment_x_l

            if np.mean(moment_z_r[: int(N_SAMPLES / 2)]) < np.mean(moment_z_r[int(N_SAMPLES / 2):]):
                moment_z_r = -moment_z_r
            if np.mean(moment_z_l[: int(N_SAMPLES / 2)]) < np.mean(moment_z_l[int(N_SAMPLES / 2):]):
                moment_z_l = -moment_z_l

            # if np.mean(moment_y_r) < 0:
            #     moment_y_r = -moment_y_r
            # if np.mean(moment_y_l) < 0:
            #     moment_y_l = -moment_y_l

            # if np.mean(moment_z_r) < 0:
            #     moment_z_r = -moment_z_r
            # if np.mean(moment_z_l) < 0:
            #     moment_z_l = -moment_z_l

            # ... Standardize
            trajectory_x_r = standardize(trajectory_x_r)
            trajectory_y_r = standardize(trajectory_y_r)
            trajectory_z_r = standardize(trajectory_z_r)

            trajectory_x_l = standardize(trajectory_x_l)
            trajectory_y_l = standardize(trajectory_y_l)
            trajectory_z_l = standardize(trajectory_z_l)

            force_x_r = standardize(force_x_r)
            force_y_r = standardize(force_y_r)
            force_z_r = standardize(force_z_r)

            force_x_l = standardize(force_x_l)
            force_y_l = standardize(force_y_l)
            force_z_l = standardize(force_z_l)

            moment_x_r = standardize(moment_x_r)
            moment_y_r = standardize(moment_y_r)
            moment_z_r = standardize(moment_z_r)

            moment_x_l = standardize(moment_x_l)
            moment_y_l = standardize(moment_y_l)
            moment_z_l = standardize(moment_z_l)

            t = np.linspace(0, 100, 1024)
            plt.plot(t, moment_z_r)
            plt.plot(t, moment_z_l)
            # plt.plot(trajectory_z_r[:, 3])
            # ax.plot3D(trajectory_y_r[100:, 3], trajectory_x_r[100:, 3], trajectory_z_r[100:, 3])


        except:
            pass

    #     break
    # break

# plt.xticks([])
# plt.yticks([])
plt.xlabel("Stance Phase (%)")
plt.ylabel("Moment (Nm)")
plt.title("Moment (z-axis)")

# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])
# ax.set_xlabel("X (mm)")
# ax.set_ylabel("Y (mm)")
# ax.set_zlabel("Z (mm)")
# ax.set_title("FM1")
plt.show()


100%|██████████| 50/50 [00:25<00:00,  1.99it/s]


In [6]:
trajectory_x_r.shape

(1024, 6)

In [7]:
# plt.plot(trajectory_x_r[:, 3])